In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/find_ideal_machine_type/find_ideal_machine_type.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/find_ideal_machine_type/find_ideal_machine_type.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/find_ideal_machine_type/find_ideal_machine_type.ipynb">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="Vertex AI Workbench notebook"> 在Vertex AI Workbench中打开
    </a>
  </td>
</table>

确定用于Vertex AI端点的理想机器类型

## 概述
本教程演示了如何根据成本和性能要求确定适合您机器学习模型的理想机器类型。

有关最佳实践的更多详细信息，请访问[这里](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#finding_the_ideal_machine_type)。

模型
本教程使用的模型是来自[TensorFlow Hub开源模型库](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4)的`BERT`模型。

## 目标

执行的步骤包括：
- 创建一个工作台笔记本，使用正在测试的机器类型。
- 从 TensorFlow Hub 下载模型。
- 创建一个本地模型并部署到本地端点。
- 对模型延迟进行基准测试。
- 清理。

成本
本教程使用Google Cloud的可计费组件：
- Vertex AI
- Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 在开始之前

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，都需要按照以下步骤进行操作。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得$300的免费信用额用于支付计算/存储成本。

2. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您正在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行视为shell命令，并且将以`$`为前缀的Python变量插入这些命令中。

设置您的项目 ID

**如果您不知道您的项目 ID**，您可以尝试使用 `gcloud` 命令获取您的项目 ID。

In [ ]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = ""  # @param {type:"string"}

时间戳

如果您正在参加现场教程会话，可能会使用共享的测试帐户或项目。为了避免在创建的资源上的用户名称冲突，请为每个实例会话创建一个时间戳，并将其附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 账户

**如果您正在使用 Google Cloud 笔记本**，您的环境已经通过验证。请跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格并按照提示进行身份验证，通过 oAuth 认证您的账户。

**否则**，请按照以下步骤操作：

1. 在 Cloud 控制台中，前往 [**创建服务账号密钥** 页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击 **创建服务账号**。

3. 在 **服务账号名称** 栏中输入一个名称，然后点击 **创建**。

4.. 在 **授予此服务账号对项目的访问权限** 部分，点击 **角色** 下拉列表。在筛选框中输入 "Vertex AI"，并选择 **Vertex AI 管理员**。在筛选框中输入 "Storage Object Admin"，并选择 **Storage Object Admin**。

5. 点击 **创建**。一个包含您密钥的 JSON 文件将下载到您的本地环境中。

6. 在下面的单元格中，将您的服务账号密钥路径作为 `GOOGLE_APPLICATION_CREDENTIALS` 变量输入，并运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

首先，您需要将模型文件上传到一个云存储桶中。利用这个模型 artifact，您可以创建 Vertex AI 模型和端点资源，以便提供在线预测。

请在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

您还可以更改`REGION`变量，该变量用于笔记本的其他部分。请确保[选择一个 Vertex AI 服务可用的地区](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。您不能使用多区域存储桶进行 Vertex AI 的训练。

In [ ]:
BUCKET_NAME = ""  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

In [ ]:
print(BUCKET_NAME)

只有当您的存储桶尚不存在时，运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -p $PROJECT_ID -l $REGION $BUCKET_NAME

最后，通过检查其内容验证对您的云存储存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

创建工作台笔记本

您将使用Google Cloud笔记本在特定的计算机类型上运行负载测试，以便对您的模型在运行在Vertex AI端点时的性能有一个良好的了解。

在这里，我们将使用`gcloud`创建笔记本，但您也可以按照[这里](https://cloud.google.com/vertex-ai/docs/workbench/user-managed/create-new#before_you_begin)的说明通过Google云控制台创建它。

In [ ]:
!gcloud notebooks instances create load-test-notebook \
--vm-image-project="deeplearning-platform-release" \
--vm-image-name="common-cpu-notebooks-v20221017-debian-10" \
--machine-type="n1-standard-8" --project=$PROJECT_ID \
--location=us-central1-a

### 打开工作台笔记本

创建笔记本后，打开该笔记本。您将在新创建的笔记本中运行其余的步骤。

### 安装Vegeta

Vegeta是一种多功能的HTTP负载测试工具，出于需要以恒定的请求速率测试HTTP服务而构建。

In [ ]:
! wget https://github.com/tsenart/vegeta/releases/download/v12.8.4/vegeta_12.8.4_linux_amd64.tar.gz

In [ ]:
! tar -xvf vegeta_12.8.4_linux_amd64.tar.gz

### 安装依赖

安装 Python 的依赖项

In [ ]:
%%writefile requirements.txt
google-cloud-aiplatform[prediction]>=1.16.0,<2.0.0
matplotlib
fastapi
contexttimer
tqdm

In [ ]:
%pip install -U --user -r requirements.txt

### 下载并提取模型

In [ ]:
! wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4?tf-hub-format=compressed -O bert.tgz
! mkdir -p bert_sentence_embedding/00001
! tar -xvf bert.tgz -C bert_sentence_embedding/00001

### 在新笔记本中设置存储桶变量

您在之前的步骤中创建了一个存储桶。因为您现在正在新的笔记本上工作，您应该重新设置存储桶变量。

In [ ]:
BUCKET_NAME = ""  # @param {type:"string"}

### 配置

为了向端点发送请求，您将创建一个虚拟请求体。

In [ ]:
# The gcs uri; remember to have a version folder under this link
# For example, GCS_URI = "gs://project/bucket/folder"
# the model should be put in "gs://project/bucket/folder/1/saved_model.pb".
GCS_URI = f"gs://{BUCKET_NAME}/bert_sentence_embedding"
REQUEST = """
{
  "instances": [
    {
      "input_word_ids": [101, 23784, 11591, 11030, 24340, 21867, 21352, 21455, 20467, 10159, 23804, 10822, 26534, 20355, 14000, 11767, 10131, 28426, 10576, 22469, 22237, 25433, 263, 28636, 12291, 119, 15337, 10171, 25585, 21885, 10263, 13706, 16046, 10112, 18725, 13668, 12208, 10104, 13336, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      "input_mask": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      "input_type_ids": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    }
  ]
}
"""

In [ ]:
!echo $GCS_URI

### 将模型复制到GCS存储桶

In [ ]:
!sudo gsutil cp -r ./bert_sentence_embedding/00001/* $GCS_URI/1/

### 记录
打开记录以查看模型的日志

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

## 顺序请求

这个测试可以测试在服务器一次只服务一个请求时的延迟（以及潜在的利用率）。您可以使用这些信息来估算单个副本可以处理多少QPS，作为配置的起点。

将LocalModel进行猴子补丁，以提供更清晰的语法...

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel


@classmethod
def create_tensorflow2(
    cls, version: str, saved_model_path: str, includes_version_subdir: bool = True
) -> LocalModel:
    version = version.replace(".", "-")
    return cls(
        serving_container_image_uri=f"us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.{version}:latest",
        serving_container_predict_route="/v1/models/default:predict",
        serving_container_health_route="/v1/models/default",
        serving_container_ports=[8501],
        serving_container_environment_variables={
            "model_name": "default",
            "model_path": saved_model_path,
        },
    )


LocalModel.create_tensorflow2 = create_tensorflow2


@classmethod
def create_pytorch(cls, version: str) -> LocalModel:
    version = version.replace(".", "-")
    return LocalModel(
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/pytorch-gpu.{version}:latest",
        serving_container_predict_route="/predictions/model",
        serving_container_health_route="/ping",
        serving_container_ports=[8080],
    )


LocalModel.create_pytorch = create_pytorch

创建本地模型并部署到本地端点。

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel.create_tensorflow2(version="2.7", saved_model_path=GCS_URI)

In [ ]:
import os.path

GPU_COUNT = 1 if os.path.exists("/dev/nvidia0") else None
print(GPU_COUNT)

In [ ]:
from contexttimer import Timer

with Timer() as timer:
    local_endpoint = local_model.deploy_to_local_endpoint(
        gpu_count=GPU_COUNT,
    )
    local_endpoint.serve()

# Actual startup time involves more than just loading the container and model, but still
# a useful number:
print(f"Startup time: {timer.elapsed}")

发送连续的请求
您将向本地端点发送多个请求，并收集延迟度量数据，这将让您很好地了解选择的机器类型在生产环境中模型的性能如何。您将可视化这些结果，并得到平均延迟时间（以毫秒为单位）。

由于这是一个变压器模型，它在 CPU 上运行速度较慢，最好使用 GPU 运行。

In [ ]:
WARMUP_REQUESTS = 10
NUM_REQUESTS = 100
PERCENTILE_POINTS = [0, 50, 95, 99, 100]
LABELS = ["min", "50", "95", "99", "max"]

import numpy as np
from contexttimer import Timer
from tqdm import tqdm

# Send some warm up requests
for _ in tqdm(range(WARMUP_REQUESTS), desc="Sending warm-up requests"):
    local_endpoint.predict(
        request=REQUEST, headers={"Content-Type": "application/json"}
    )

# Send sequential requests
latencies = []
for _ in tqdm(range(NUM_REQUESTS), desc="Sending requests"):
    with Timer(factor=1000) as timer:
        local_endpoint.predict(
            request=REQUEST, headers={"Content-Type": "application/json"}
        )
    latencies.append(timer.elapsed)

percentiles = np.percentile(latencies, PERCENTILE_POINTS)

In [ ]:
from matplotlib import pyplot as plt

plt.hist(latencies, bins=50, density=True)
plt.xlabel("Latency (ms)")
plt.show()

for p, v in zip(["min", "50", "95", "99", "max"], percentiles):
    print(f"{p}: {v:0.1f}")

print(f"mean: {np.average(latencies):0.1f}")

发送并发请求

上面的练习为每个请求的延迟提供了一个很好的基准，但并不能表明模型在生产环境中处理并发请求时的表现。例如，当机器的资源耗尽时，延迟可能会降低。为了找到一个能够有效处理多个并发请求的理想机器类型，我们将使用 `vegeta`。

In [ ]:
BATCH_SIZE = 1
REQUEST_FILE = "request.json"

import json

instance = json.loads(REQUEST)["instances"][0]
# Row-based encoding
with open(REQUEST_FILE, "w") as f:
    json.dump({"instances": [instance] * BATCH_SIZE}, f)

# Column-based encoding (more efficient for some models)
inputs = {feature: [values] * BATCH_SIZE for feature, values in instance.items()}
with open("request_cols.json", "w") as f:
    json.dump({"inputs": inputs}, f)

In [ ]:
URL = f"http://localhost:{local_endpoint.assigned_host_port}{local_endpoint.serving_container_predict_route}"
URL

In [ ]:
!curl http://localhost:{local_endpoint.assigned_host_port}{local_endpoint.serving_container_health_route}

In [ ]:
!curl -X POST http://localhost:{local_endpoint.assigned_host_port}{local_endpoint.serving_container_predict_route} -d @request.json

In [ ]:
DURATION = "100s"

! for i in 1 2 3 4; do \
    echo "POST {URL}" | \
   ./vegeta attack -header "Content-Type: application/json" -body {REQUEST_FILE} -rate ${{i}} -duration {DURATION} | \
   tee report-${{i}}.bin | \
   ./vegeta report --every=60s; \
  done

In [ ]:
! for f in `ls *.bin`; do \
    ./vegeta report --type=json ${{f}} > ${{f}}.json; \
  done

In [ ]:
import glob
import json
import re

throughput, p99, avg = {}, {}, {}
for fn in glob.glob("report-*.bin.json"):
    with open(fn) as f:
        data = json.load(f)
    qps = int(re.search(r"report-(\d+).bin.json", fn).group(1))
    throughput[qps] = data["throughput"]
    p99[qps] = data["latencies"]["99th"] / 1000000
    avg[qps] = data["latencies"]["mean"] / 1000000

In [ ]:
import matplotlib.pyplot as plt

points = sorted(p99.items(), key=lambda item: item[0])
x, y = zip(*points)
plt.plot(x, y, "-o")
plt.xlabel("Target QPS")
plt.ylabel("P99 Latency (ms)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

points = sorted(throughput.items(), key=lambda item: item[0])
x, y = zip(*points)
plt.plot(x, y, "-o")
plt.xlabel("Target QPS")
plt.ylabel("Actual QPS")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

points = sorted(avg.items(), key=lambda item: item[0])
x, y = zip(*points)
plt.plot(x, y, "-o")
plt.xlabel("Target QPS")
plt.ylabel("Average Latency (ms)")
plt.show()

我们可以估计单个副本可以处理的并发请求数量：

$num\_concurrent\_requests = \frac{qps}{avg\_latency_{qps}}$

In [ ]:
QPS = 2

num_concurrent_requests = QPS / avg[QPS]
num_concurrent_requests

正如您所见，这种模型在该类型的机器上表现不佳。尝试不同的机器类型配置，或添加GPU，看看结果如何变化。

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
!gsutil rm -r $GCS_URI/*

以下命令将删除用于测试的工作台笔记本实例。在继续之前，请保存您的所有工作。

In [ ]:
!gcloud notebooks instances delete load-test-notebook